## Desafio de Análise de Dados: Empréstimos de Acervos das Bibliotecas da UFRN

Neste desafio do #7DaysOfCode, o objetivo é analisar os dados de empréstimos dos acervos do sistema de bibliotecas da UFRN, utilizando as ferramentas do Python como o Pandas. A partir dessa análise, será possível extrair insights valiosos sobre o comportamento dos empréstimos ao longo dos últimos anos, identificar padrões e formular hipóteses para melhorar a gestão e os processos das bibliotecas.

A análise exploratória de dados (EDA) será a principal metodologia utilizada. A EDA é fundamental para compreender as tendências nos dados, explorar informações sobre os processos, usuários e recursos das bibliotecas, além de gerar novas perguntas que podem orientar decisões mais assertivas na gestão do acervo. Durante o processo, vamos focar em questões como:

A quantidade de empréstimos tem aumentado ou diminuído ao longo dos anos?

Quais bibliotecas apresentam o maior número de empréstimos?

Quais são os temas mais e menos requisitados pelos usuários?

Para atingir esses objetivos, o trabalho será dividido em várias etapas, desde a coleta e transformação dos dados até a criação de visualizações e a exportação de tabelas para um formato utilizável em diferentes contextos. O Pandas será utilizado para importar dados de diferentes fontes, realizar transformações necessárias e, por fim, gerar insights que possam ser apresentados de forma clara e eficaz.

Com esses dados, será possível entender melhor o funcionamento do sistema de bibliotecas e fornecer informações para tomadas de decisão mais informadas, visando melhorar a experiência dos usuários e a utilização dos recursos disponíveis.

In [ ]:
# Instalação das bibliotecas necessárias
%pip install pandas
%pip install fastparquet

In [2]:
# Importação das bibliotecas necessárias
import pandas as pd

### Coletando os dados de empréstimos (2010 a 2020)

In [6]:
anos = range(2010, 2021)
semestres = [1, 2]
arquivos = []

for ano in anos:
    for semestre in semestres:
        if ano == 2020 and semestre == 2:
            continue  # Esse arquivo não existe
        url = f"https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/blob/main/Dia_1-Importando_dados/Datasets/dados_emprestimos/emprestimos-{ano}{semestre}.csv?raw=true"
        try:
            df = pd.read_csv(url)
            arquivos.append(df)
        except Exception as e:
            print(f"Erro ao carregar {url}: {e}")

### Concatenando os DataFrames

In [7]:
emprestimos = pd.concat(arquivos, ignore_index=True)

### Verificando Duplicatas

In [11]:
duplicatas = emprestimos[emprestimos.duplicated()]
print(f"Total de duplicatas encontradas: {len(duplicatas)}")

Total de duplicatas encontradas: 0


### Excluindo Duplicatas

In [9]:
emprestimos = emprestimos.drop_duplicates()

In [10]:
# Exibindo as primeiras linhas do DataFrame
emprestimos.head(10)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO
1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2.008023e+09,ALUNO DE GRADUAÇÃO
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO
3,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,2.007211e+08,ALUNO DE GRADUAÇÃO
8,709688,2007027500,2010/01/19 15:59:26.464000000,2010/01/04 08:27:33.569000000,2010/02/03 13:06:30.662000000,2.009046e+09,ALUNO DE GRADUAÇÃO
9,709689,L157395,2010/01/19 15:59:26.447000000,2010/01/04 08:27:33.596000000,2010/02/03 13:06:43.125000000,2.009046e+09,ALUNO DE GRADUAÇÃO
10,709690,2007019312,NaN,2010/01/04 08:49:57.857000000,2010/01/13 09:58:25.443000000,2.007014e+08,ALUNO DE GRADUAÇÃO
11,709691,2008046357,2010/01/19 11:50:13.404000000,2010/01/04 09:02:30.576000000,2010/02/02 12:13:36.132000000,2.005075e+08,ALUNO DE GRADUAÇÃO
12,709692,L187962,NaN,2010/01/04 09:07:06.131000000,2010/06/22 18:55:04.000000000,2.004044e+08,ALUNO DE GRADUAÇÃO
13,709693,L192776,NaN,2010/01/04 09:07:06.159000000,2010/06/22 18:54:46.000000000,2.004044e+08,ALUNO DE GRADUAÇÃO


### Importando os Dados de exemplares

In [12]:
dados_exemplares = pd.read_parquet('https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_1-Importando_dados/Datasets/dados_exemplares.parquet')
dados_exemplares

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15
...,...,...,...,...,...,...,...
568580,1353472,2021011150,Acervo Circulante,Biblioteca Setorial do Núcleo de Educação da I...,REGULAR,951,268231
568581,1353473,2019013454,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,997,268233
568582,1353474,2019012811,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,987,268234


In [13]:
todos_emprestimos = emprestimos.merge(dados_exemplares, on="codigo_barras", how="inner")
todos_emprestimos.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO,13259,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,4225
1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2.008023e+09,ALUNO DE GRADUAÇÃO,70865,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,25009
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO,195347,Acervo Circulante,Biblioteca Setorial Prof. Rodolfo Helinski - E...,REGULAR,640,75019
3,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,2.007211e+08,ALUNO DE GRADUAÇÃO,131639,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,609,44494
4,709688,2007027500,2010/01/19 15:59:26.464000000,2010/01/04 08:27:33.569000000,2010/02/03 13:06:30.662000000,2.009046e+09,ALUNO DE GRADUAÇÃO,225646,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,686,91736
